# **Recommendation Systems**

In this part, we are going to implement a Recommendation System for properties for sale in Argentina. Based on a property id, the system is going to recommend 10 other properties that are similar, based on cosine similarity, to the specified property.

In [1]:
! pip install pyspark seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=88259ef1c6e5e9662131ad39ca34f8dde443fc7b4b6669f6054a3e9d7e6f8ddd
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# libraries

import sklearn
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from scipy.spatial.distance import cosine
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.window import Window

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
google_drive_path = "/content/gdrive/MyDrive/Colab Notebooks/Project/"

Mounted at /content/gdrive


In [4]:
# we've configured the Spark session to handle large datasets with high memory and parallelism settings
spark = SparkSession.builder.appName("Project511_RS").master("local[*]") \
          .config("spark.driver.memory", "10g") \
          .config("spark.executor.memory", "10g") \
          .config("spark.driver.maxResultSize", "2g") \
          .config("spark.sql.shuffle.partitions", "200") \
          .config("spark.default.parallelism", "100") \
          .getOrCreate()

# Read Datasets

We are going to load two versions of your dataset: one with numeric representations (for similarity calculations) and another with human-readable property characteristics (for presenting recommendations).

Read the latest version of our sale df with all the preprocessing or other manipulation we did to the data.

In [5]:
sale_df = spark.read.format("parquet").load(google_drive_path + "sale_ml.parquet")

In [6]:
sale_df.printSchema()

root
 |-- price_bucket: double (nullable = true)
 |-- start_year: integer (nullable = true)
 |-- end_year: integer (nullable = true)
 |-- is_active: integer (nullable = true)
 |-- l3_one_hot: vector (nullable = true)
 |-- l2_one_hot: vector (nullable = true)
 |-- property_one_hot: vector (nullable = true)
 |-- encoded_start_month: vector (nullable = true)
 |-- encoded_end_month: vector (nullable = true)
 |-- encoded_start_day_of_week: vector (nullable = true)
 |-- encoded_end_day_of_week: vector (nullable = true)
 |-- geo_cluster: integer (nullable = true)
 |-- rooms_scaled: double (nullable = true)
 |-- bedrooms_scaled: double (nullable = true)
 |-- bathrooms_scaled: double (nullable = true)
 |-- surface_total_scaled: double (nullable = true)
 |-- scaled_final_features: vector (nullable = true)



This df (sale_df) does not have the description of the properties in textual representation, but only in numeric representation, which we converted in the Text Analytics file, so we are going to read the processed df without any encoded or scaled feature, but only the actual - readable characteristics of the properties. Both datasets have the same rows and data in the same order, but in order to give the description of the property when we recommending other properties, we are going to merge the two datasets on the id column that we will create afterwards.

In [7]:
df = spark.read.format("parquet").load(google_drive_path + "ar_properties_processed.parquet")

This is the dataset with the descriptions and all the info not encoded.

In [8]:
df.show()

+----------+----------+--------------------+--------------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+----------------+--------------+--------------------+------------------+
|start_date|  end_date|                  l2|                  l3|rooms|bedrooms|bathrooms|surface_total|surface_covered|               title|         description|   property_type|operation_type|         coordinates|    amount_in_euro|
+----------+----------+--------------------+--------------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+----------------+--------------+--------------------+------------------+
|2019-12-29|2020-02-11|Bs.As. G.B.A. Zon...|               Pilar|    4|       3|        2|        140.0|          140.0|Susana Aravena Pr...|Excelente propied...|           House|          Rent|[-34.879692077636...|             533.8|
|2019-12-29|2020-04-04|Bs.As. G.B.A. Zon...|             Esc

We are going to do some preprocessing in order to have the consistent version of it (we did some changes during the machine learning task).

In [9]:
df = df.filter(col("operation_type") == "Sale")

In [10]:
df.count()

210341

In [11]:
df = df.filter((col("amount_in_euro") <= 9000000000) & (col("amount_in_euro") > 11))

In [12]:
df.count()

210339

We are going to add the unique property id to both datasets using pyspark window function, in order to merge them later.

# Index

## Readable Dataset

In [13]:
# add column called 'id' that contains row numbers from 1 to n
w = Window().orderBy(lit('A'))
df = df.withColumn('id', row_number().over(w))

In [14]:
df.show()

+----------+----------+--------------------+--------------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+-------------+--------------+--------------------+------------------+---+
|start_date|  end_date|                  l2|                  l3|rooms|bedrooms|bathrooms|surface_total|surface_covered|               title|         description|property_type|operation_type|         coordinates|    amount_in_euro| id|
+----------+----------+--------------------+--------------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+-------------+--------------+--------------------+------------------+---+
|2019-12-29|2020-06-14|Bs.As. G.B.A. Zon...|              Moreno|    2|       2|        1|        300.0|          260.0|Azucena Villaflor...|Excelente oportun...|          Lot|          Sale|[-34.879692077636...| 7064.999999999999|  1|
|2019-12-29|2019-12-29|             Mendoza|           L

## Sale Dataset - Encoded

In [15]:
sale_df = sale_df.withColumn('id', row_number().over(w))

In [16]:
sale_df.show()

+------------+----------+--------+---------+-------------+--------------+----------------+-------------------+-----------------+-------------------------+-----------------------+-----------+-------------------+--------------------+-------------------+--------------------+---------------------+---+
|price_bucket|start_year|end_year|is_active|   l3_one_hot|    l2_one_hot|property_one_hot|encoded_start_month|encoded_end_month|encoded_start_day_of_week|encoded_end_day_of_week|geo_cluster|       rooms_scaled|     bedrooms_scaled|   bathrooms_scaled|surface_total_scaled|scaled_final_features| id|
+------------+----------+--------+---------+-------------+--------------+----------------+-------------------+-----------------+-------------------------+-----------------------+-----------+-------------------+--------------------+-------------------+--------------------+---------------------+---+
|         4.0|      2020|    2020|        0|(4,[0],[1.0])|(27,[2],[1.0])|   (9,[0],[1.0])|     (11,[7],

Final_features are the numeric representation of each property's description so we have scaled them. This is necessary as different features can have varying ranges and scaling them helps in normalizing these differences.
This step prepares the dataset for more effective similarity calculations by ensuring that all features contribute equally, improving the quality of recommendations in the recommendation system.

In [17]:
# reorder the features in order to have id first
sale_df = sale_df.select("id", "scaled_final_features", "start_year", "end_year", "is_active", "l3_one_hot", "l2_one_hot", "property_one_hot", \
               "encoded_start_month", "encoded_end_month", "encoded_start_day_of_week",  "encoded_end_day_of_week", "geo_cluster", "rooms_scaled", \
               "bedrooms_scaled", "bathrooms_scaled", "surface_total_scaled", "price_bucket")

In [18]:
sale_df.show()

+---+---------------------+----------+--------+---------+-------------+--------------+----------------+-------------------+-----------------+-------------------------+-----------------------+-----------+-------------------+--------------------+-------------------+--------------------+------------+
| id|scaled_final_features|start_year|end_year|is_active|   l3_one_hot|    l2_one_hot|property_one_hot|encoded_start_month|encoded_end_month|encoded_start_day_of_week|encoded_end_day_of_week|geo_cluster|       rooms_scaled|     bedrooms_scaled|   bathrooms_scaled|surface_total_scaled|price_bucket|
+---+---------------------+----------+--------+---------+-------------+--------------+----------------+-------------------+-----------------+-------------------------+-----------------------+-----------+-------------------+--------------------+-------------------+--------------------+------------+
|  1| (17000,[0,1,2,3,6...|      2020|    2020|        0|(4,[0],[1.0])|(27,[2],[1.0])|   (9,[0],[1.0])|

In [19]:
sale_df.count()

210339

# Feature Vector Construction

The goal of constructing a feature vector is to combine all relevant features into a single vector that represents each property. This vector is then utilized for calculating similarities between properties in the recommendation system.

This process is vital for ensuring that the properties are represented in a way that enhances the effectiveness and accuracy of the recommendation system, leveraging both numerical and categorical data to identify the most similar properties based on a variety of characteristics.

In [20]:
# list with both scaled numeric features and categorical features that have been one-hot encoded
feature_columns = [
    'scaled_final_features', 'price_bucket', 'l3_one_hot', 'l2_one_hot', 'property_one_hot',
    'encoded_start_month', 'encoded_end_month', 'encoded_start_day_of_week', 'encoded_end_day_of_week',
    'rooms_scaled', 'bedrooms_scaled', 'bathrooms_scaled', 'surface_total_scaled']

# VectorAssembler is a transformer that combines a given list of columns into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
sale_df = assembler.transform(sale_df) # trasform df

# Sample

Unfortunately, due to constraints in computational resources and to ensure the system remains performant, we decided to proceed with only 10% of the full dataset.

:(

Fortunately, as we will see later on, the results weren't that disappointing.

:)

In [21]:
sale_df = sale_df.sample(fraction=0.1, seed=1234)

In [22]:
sale_df.select("id").show()

+---+
| id|
+---+
| 31|
| 36|
| 38|
| 47|
| 51|
| 55|
| 61|
| 63|
| 88|
| 89|
| 93|
| 98|
|121|
|132|
|134|
|142|
|143|
|165|
|171|
|174|
+---+
only showing top 20 rows



**Example:**

For demonstration purposes, we select an example property from the sampled dataset. Recommendations for this property will showcase how the system can identify and suggest similar properties based on the learned features.

In [23]:
property_id = sale_df.first()["id"]

In [24]:
property_id

31

# Normalization

We have to ensure that our feature vectors are normalized. This simplifies the cosine similarity to just a dot product since cosine similarity between two normalized vectors A and B is just their dot product. This simplifies and speeds up the computation significantly.

In [25]:
# initialize the Normalizer which will be used to normalize the feature vectors
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
sale_df = normalizer.transform(sale_df) # transform sale df


In [26]:
#  caching the df to optimize performance as it will be accessed multiple times during the recommendation process
sale_df.cache()

DataFrame[id: int, scaled_final_features: vector, start_year: int, end_year: int, is_active: int, l3_one_hot: vector, l2_one_hot: vector, property_one_hot: vector, encoded_start_month: vector, encoded_end_month: vector, encoded_start_day_of_week: vector, encoded_end_day_of_week: vector, geo_cluster: int, rooms_scaled: double, bedrooms_scaled: double, bathrooms_scaled: double, surface_total_scaled: double, price_bucket: double, features: vector, normFeatures: vector]

In [27]:
# repartitioning the df by 'id' to optimize the layout of data across the cluster, improving the efficiency of the query operations that follow
sale_df = sale_df.repartition(50, "id")

# Similarity-Based Recommender

Since we are not dealing with user interactions but rather with item similarity, traditional collaborative filtering techniques like ALS (Alternating Least Squares) may not be suitable. Instead, we can focus on content-based filtering using the feature vectors we've constructed. By focusing on content-based filtering, the system can make precise recommendations based on detailed property features.


In [28]:
# UDF for calculating cosine similarity between two feature vectors
def cosine_similarity(x, y):
    return float(1 - cosine(np.array(x), np.array(y))) # convert vectors to numpy arrays and compute the cosine similarity


In [29]:
# udf
cosine_similarity_udf = udf(cosine_similarity, FloatType())

Function to recommend properties based on similarity of feature vectors

In [30]:
def recommend_properties(property_id, num_recommendations=10):
    try:
        # retrieve the feature vector for the specified property
        target_feature_vector = sale_df.filter(col("id") == property_id).select("features").first()
        if target_feature_vector is None:
            return f"No property found with ID {property_id}"

        # convert the sparse vector to a numpy array for similarity computation
        target_feature_vector = target_feature_vector["features"].toArray()

        # broadcast the target feature vector to all nodes to minimize data transfer during similarity calculations
        broadcast_vector = spark.sparkContext.broadcast(target_feature_vector)

        # calculate the similarity of the broadcasted vector with all properties' feature vectors
        similarities = sale_df.withColumn("similarity", cosine_similarity_udf(col("features"), lit(broadcast_vector.value)))

        # get the top properties ordered by their similarity scores, excluding the property itself
        recommendations = similarities.orderBy("similarity", ascending=False).limit(num_recommendations + 1)
        recommendations = recommendations.filter(col("id") != property_id)

        return recommendations.select("id", "similarity")

    # handle exceptions
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [31]:
recommended_properties = recommend_properties(property_id).cache()

In [32]:
recommended_properties.show()

+------+----------+
|    id|similarity|
+------+----------+
| 15503|  0.910231|
| 23616| 0.8911548|
| 20686| 0.8837744|
|103114| 0.8835265|
|153171|0.87793225|
|107779| 0.8747517|
|115588| 0.8681503|
| 19007| 0.8631911|
|181483|0.86172354|
|140950| 0.8527636|
+------+----------+



As we can observe, the scores of the recommended properties are quite high, meaning that the recommendation system, can efficiently suggest similar properties, compared to the specified one. In a real case scenario, such similarity metrics can help users discover properties that match their preferences or needs closely.

# Merge Datasets

We merge the two datasets, in order to get a detailed description in a readable representation for each recommended property.

In [33]:
merged_df = recommended_properties.join(df, "id")

In [34]:
merged_df.show()

+------+----------+----------+----------+--------------------+---------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+-------------+--------------+--------------------+--------------+
|    id|similarity|start_date|  end_date|                  l2|             l3|rooms|bedrooms|bathrooms|surface_total|surface_covered|               title|         description|property_type|operation_type|         coordinates|amount_in_euro|
+------+----------+----------+----------+--------------------+---------------+-----+--------+---------+-------------+---------------+--------------------+--------------------+-------------+--------------+--------------------+--------------+
| 15503|  0.910231|2020-06-20|2020-06-20|     Capital Federal|   Barrio Norte|    3|       2|        1|         78.0|           62.0|Excelente inmuebl...|RETASADO OPORTUNI...|    Apartment|          Sale|[-34.879692077636...|      137080.0|
| 19007| 0.8631911|2020-03-15|2020-0

Recommendation Score: It confirms that the system can identify properties similar to a given reference, validated by similarity scores.

Information Richness: The extensive details accompanying each recommendation enhance transparency and trust, enabling users to understand why these properties were suggested.


Last but not least,we implemented one more recommendation system. The main idea of this, is to create "ground truth" by using KMeans Clustering, train on training set and get recommendations and then test that the recommendations are in the same cluster as the specified property id. The results may not be reliable, as we do not have an actual label.